In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm 
from transformers import ViltProcessor, ViltModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
file_path = "/users/PAS2136/rayees/CV4A - Efficient Probing/ProbingSubsets/probing_14500_ratio.csv"
df = pd.read_csv(file_path)
cols = ['ImageFilePath', 'ScientificName']
df = df[cols]
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Notiophilus biguttatus
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Notiophilus biguttatus


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "dandelin/vilt-b32-mlm"

model = ViltModel.from_pretrained(model_name).eval().to(device)

processor = ViltProcessor.from_pretrained(model_name)


In [5]:
def extract_features(image_path, taxa) :
    
    image = Image.open(image_path).convert("RGB")
    
    text = f"This is an image of a beetle, of species : {taxa}"
    
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()
    

In [ ]:
X = np.vstack([extract_features(img, taxa) for img, taxa in tqdm(zip(df["ImageFilePath"], df["ScientificName"]), desc="Processing ....") ])

seed = 99

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X, y, df_indices, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)


Processing ....: 0it [00:00, ?it/s]

In [ ]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [ ]:

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


In [ ]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

In [ ]:
metrics_df

In [ ]:

basedir = '/users/PAS2136/rayees/CV4A - Efficient Probing/BritishCarabids_14500_ratio/'


In [ ]:

test_df.to_csv(f"{basedir}/6.ViLT-species.csv", index=False)

metrics_df.to_csv(f"{basedir}/6.ViLT-species-metrics.csv", index=False)

